In [1]:
#!python -m grpc_tools.protoc -I. --python_out=. --grpc_python_out=. poam.proto

In [2]:
import grpc
import json
import base64
import poam_pb2
import poam_pb2_grpc
server_address = "localhost:50051"
proving_image_id =  [
            2948630792,
            2482136711,
            1512618692,
            3197082418,
            820963868,
            463243005,
            2815130045,
            3835354657
            ]


In [3]:
def proof_request(input_file, method_payload, chained = False, proving_image_id = proving_image_id, server_address = "localhost:50051")->dict:
    # Load input data from JSON file
    with open(input_file, "r") as infile:
        input_data = json.load(infile)
    
    # Create a channel and a stub
    channel = grpc.insecure_channel(server_address)
    stub = poam_pb2_grpc.VerifiableProcessingServiceStub(channel)
    
    if not chained:
        # Construct the ProveRequest message using data from JSON
        request = poam_pb2.ProveRequest(
            poam_metadata=poam_pb2.PoamMetadata(
                current_image_id=proving_image_id,
            ),
            method_payload=method_payload
        )
    else:
        request = poam_pb2.ProveRequest(
            poam_metadata=poam_pb2.PoamMetadata(
                previous_proof  = poam_pb2.Proof(
                    image_id = input_data["proof"]["imageId"],
                    receipt = base64.b64decode(input_data["proof"]["receipt"].encode("utf-8"))
                ),
                current_image_id=input_data["proof"]["imageId"]
            ),
            method_payload=method_payload
        )
    # Call the prove method
    try:
        response = stub.prove(request)
        # Convert response to a dictionary
        
        image_id = response.proof_response.image_id
        receipt = base64.b64encode(response.proof_response.receipt)
        #
        response_dict = {
            "verificationValue": float(response.public_output),
            "proof":{
                "imageId": list(image_id),
                "receipt": receipt.decode("utf-8")
            }
        }
        return response_dict
    except grpc.RpcError as e:
        print("gRPC error:", e.details())

In [4]:
def compose_request(proof_chain, server_address = "localhost:50051")->dict:
    # Create a channel and a stub
    channel = grpc.insecure_channel(server_address)
    stub = poam_pb2_grpc.VerifiableProcessingServiceStub(channel)

    # Construct the ProveRequest message using data from JSON
    request = poam_pb2.CompositionRequest(
        proof_chain = proof_chain,
    )
    
    try:
        response = stub.compose(request)
        # Convert response to a dictionary
        image_id = response.proof_response.image_id
        receipt = base64.b64encode(response.proof_response.receipt)
        #
        response_dict = {
            "verificationValue": 0.0,
            "proof":{
                "imageId": list(image_id),
                "receipt": receipt.decode("utf-8")
            }
        }
        return response_dict
    except grpc.RpcError as e:
        print("gRPC error:", e.details())

In [8]:
def combined_request(experiments_file, server_address = "localhost:50051")->dict:
    # Load input data from JSON file
    with open(experiments_file, "r") as infile:
        input_data = json.load(infile)


    channel = grpc.insecure_channel(server_address)
    stub = poam_pb2_grpc.VerifiableProcessingServiceStub(channel)
    # Construct the ProveRequest message using data from JSON
    request = poam_pb2.CombinedRequest(
        method_payload = str(input_data)
    )
    
    try:
        response = stub.combined(request)
        # Convert response to a dictionary
        image_id = response.proof_response.image_id
        receipt = base64.b64encode(response.proof_response.receipt)
        #
        response_dict = {
            "verificationValue": float(response.public_output),
            "proof":{
                "imageId": list(image_id),
                "receipt": receipt.decode("utf-8")
            }
        }
        return response_dict
    except grpc.RpcError as e:
        print("gRPC error:", e.details())

In [6]:
def verify_request(proof, verification_value = 0.0, server_address = "localhost:50051")->tuple:
    # Create a channel and a stub
    channel = grpc.insecure_channel(server_address)
    stub = poam_pb2_grpc.VerifiableProcessingServiceStub(channel)
    # Construct the ProveRequest message using data from JSON
    request = poam_pb2.VerifyRequest(
        verification_value = verification_value,
        proof = proof
    )
    #poam_pb2.Proof(image_id=input_data1["proof"]["imageId"], receipt=base64.b64decode(input_data1["proof"]["receipt"].encode("utf-8")))
    # Call the prove method
    try:
        response = stub.verify(request)
        # Convert response to a dictionary
        #print(response.is_valid_executed)
        #print(response.public_output)
        return (response.is_valid_executed, response.public_output)
        # Write response to a JSON file
    except grpc.RpcError as e:
        print("gRPC error:", e.details())